# Project 1: #

### Scraping Twitter Fun  ###

In [26]:
import tweepy, datetime, time
from collections import defaultdict
import numpy as np
import pickle
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import subjectivity
from nltk.sentiment import SentimentAnalyzer
from nltk.sentiment.util import *
import re
from textblob import TextBlob
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import csv

In [2]:
consumer_key = ["QZPN8u1naWkwtxadO2pImqrnG",
               "xJm6LpUdraMvhsKWTN1EeS8XA",
               "sKHaUy6PxtzHNiussFhTTNXYp",
               "tOmzZqdAiJSBDrthx3wpEWR3n",
               "Y0qpzfZ2WyRsKPJaPdwnRcPMe",
               "6kMkhoJs79NHlPSRflN8upMxe"]
consumer_secret = ["EfTVgBBXsFfyKbRiyDj4O6UwwZRCJEjxCENRrT6F9jLIEVsb7q",
                  "gqCh5tlL5pJYJmOSuuVc7ZiXW76v1XABeRsXuk4USFxvmAC5Cc",
                  "wxRbtA7WE6OY9DvRHkGQX20aA9RzegKkYoOha6h5YPapvsF3v2",
                  "B91kG4Gjs4QOMf51iBIwdvezets0CyaDt7hWasHRiKTGPBjErp",
                  "IRaXFODzl5ooxM8bAccNkg5BQKO1ti3arUyglBStwQ3X1qAcvI",
                  "U0Gb46YyTxg6HDUJsiEHxbdHMIekAj4TMMx2iGXL0GW40VegpK"]
access_key = ["836420062991028224-VFazjwuMNYNkr0QIpKxTEvu64Q951Be",
             "836420062991028224-Z7ZvKgy6wZIiMLHKAQ0RJ9IFVc0YSZT",
             "836420062991028224-Qu3UBibTbljWsfhd2LNLMwIE9AFoKz6",
             "717450360344993793-joDXgYT22q2CgAoGry4cnl6FKGxBv4V",
             "717450360344993793-UHGgXNFp6YCibhG38nH41dEdxd3GGdn",
             "717450360344993793-KVrFXsxGptS6DcnJ5jPFQyRAtF22dWk"]
access_secret = ["Hbz7ZQ2s7GubGP1dkrmHH58gSjLDUs0WVezZbSoo3ADYU",
                "hSZYvtxCFjHe4NK62JSQI4xxvb95mgpgICz4JLM5xAJ66",
                "Ph3vPV68vCOgW0blgRYm6rqHhhu0xeTzrHZ0EGxSo4ehM",
                "KirJ3Zf64A9tAeRHzZBvq4sLr5M7GYMmcUzT57qw2Ecej",
                "LzBdVGxxgsXfCzdrIq7JRxAAveainZsXN7V0EaPcz6Sm2",
                "1CasQyAqxThQDgWMArGeFvtTVTZmls2gs8HnRGDUS4Rpv"]

In [3]:
auth = tweepy.OAuthHandler(consumer_key[0], consumer_secret[0])
auth.set_access_token(access_key[0], access_secret[0])
api = tweepy.API(auth)

In [4]:
top_twitter_users = ["@katyperry", "@justinbieber", "@BarackObama", "@taylorswift13", "@rihanna",
                    "@TheEllenShow", "@ladygaga", "@jtimberlake", "@KimKardashian",
                    "@selenagomez", "@jimmyfallon", "@ArianaGrande",
                    "@ddlovato", "@MileyCyrus", "@kanyewest", "@realDonaldTrump",
                    "@KingJames", "@Oprah", "@Adele", "@Drake", "@KevinHart4real",
                    "@RyanSeacrest", "@JLo", "@NICKIMINAJ"]

In [33]:
top_twitter_users_lowered = [x.lower() for x in top_twitter_users]

In [6]:
twitter_users_idx = defaultdict(int)
for i in range(len(top_twitter_users)):
    twitter_users_idx[top_twitter_users[i]]=i

In [8]:
def update_keys(iteration):
    auth = tweepy.OAuthHandler(consumer_key[iteration], consumer_secret[iteration])
    auth.set_access_token(access_key[iteration], access_secret[iteration])
    return tweepy.API(auth)

In [9]:
iteration=1
i=-1

In [10]:
all_tweets = defaultdict(list)
for user in top_twitter_users:
    i+=1
    if i==4 or i==8 or i==12 or i==16 or i==20:
        api = update_keys(iteration)
        iteration+=1
    qr = tweepy.Cursor(api.user_timeline, id=user)
    for tw in qr.items():
        created = tw.created_at
        if created < datetime.datetime(2015, 1, 1, 0, 0):
            break
        all_tweets[user].append({'text': tw.text, 'date': tw.created_at})

In [13]:
pickle.dump(all_tweets, open("all_tweets.p", "wb"))

In [14]:
all_tweets = pickle.load(open("all_tweets.p", "rb"))

### Adding Recipient of Tweet to Data###

In [16]:
def get_recipient(tweet):
    return re.findall(r'\B\@\w+\b', tweet)

In [17]:
for user in top_twitter_users:
    for i in range(len(all_tweets[user])):
        all_tweets[user][i]['recipient'] = get_recipient(all_tweets[user][i]['text'])

### Removing Tweets Not Sent to Top Users###

In [45]:
good_tweets = defaultdict(list)
for user in top_twitter_users:
    for tweet in all_tweets[user]:
        for recipient in tweet['recipient']:
            tweet2=tweet.copy()
            if (recipient.lower() in top_twitter_users_lowered) and (not recipient.lower()==user.lower()):
                tweet2['recipient'] = [recipient]
                good_tweets[user].append(tweet2)
                break

### Doing Some Sentiment Analysis ###

In [48]:
def processTweet(tweet_text):
    tweet_text = tweet_text.lower()
    tweet_text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','URL',tweet_text)
    tweet_text = re.sub('@[^\s]+','AT_USER',tweet_text)
    tweet_text = re.sub('[\s]+', ' ', tweet_text)
    tweet_text = re.sub(r'#([^\s]+)', r'\1', tweet_text)
    tweet_text = tweet_text.strip('\'"')
    return tweet_text

def get_tweet_sentiment(tweet):
    analysis = TextBlob(processTweet(tweet))
    if analysis.sentiment.polarity > 0:
        return 1 # positive
    elif analysis.sentiment.polarity == 0:
        return 0 # neutral
    else:
        return -1 # negative

In [49]:
for user in top_twitter_users:
    for i in range(len(good_tweets[user])):
        good_tweets[user][i]['sentiment'] = get_tweet_sentiment(good_tweets[user][i]['text'])

### Making into matrix for simplicity

In [50]:
good_tweets_matrix = []
for user in top_twitter_users:
    for tweet in good_tweets[user]:
        for recipient in tweet['recipient']:
            good_tweets_matrix.append([user, tweet['date'], recipient, tweet['text'], tweet['sentiment']])
for i in range(len(good_tweets_matrix)):
    txt = good_tweets_matrix[i][3]
    good_tweets_matrix[i][3] = txt.encode('ascii',errors='ignore')

In [51]:
my_df = pd.DataFrame(good_tweets_matrix)
my_df.to_csv('good_tweets_matrix.csv', index=False, header=False)

### Aggregating Communication ###

In [52]:
countPos = np.zeros(len(top_twitter_users))
countNeutral = np.zeros(len(top_twitter_users))
countNeg = np.zeros(len(top_twitter_users))
for user in top_twitter_users:
    for tweet in good_tweets[user]:
        for recipient in tweet['recipient']:
            if tweet['sentiment']==1:
                countPos[twitter_users_idx[recipient]]+=1
            elif tweet['sentiment']==-1:
                countNeg[twitter_users_idx[recipient]]+=1
            else:
                countNeutral[twitter_users_idx[recipient]]+=1

In [53]:
for i in range(len(top_twitter_users)):
    print(top_twitter_users[i])
    print("Total Positive Responses: "+str(countPos[i]))
    print("Total Negative Responses: "+str(countNeg[i]))
    print("Total Neutral Responses: "+str(countNeutral[i])+"\n")

@katyperry
Total Positive Responses: 124.0
Total Negative Responses: 28.0
Total Neutral Responses: 86.0

@justinbieber
Total Positive Responses: 20.0
Total Negative Responses: 7.0
Total Neutral Responses: 20.0

@BarackObama
Total Positive Responses: 16.0
Total Negative Responses: 3.0
Total Neutral Responses: 12.0

@taylorswift13
Total Positive Responses: 22.0
Total Negative Responses: 2.0
Total Neutral Responses: 12.0

@rihanna
Total Positive Responses: 6.0
Total Negative Responses: 3.0
Total Neutral Responses: 4.0

@TheEllenShow
Total Positive Responses: 63.0
Total Negative Responses: 11.0
Total Neutral Responses: 33.0

@ladygaga
Total Positive Responses: 12.0
Total Negative Responses: 3.0
Total Neutral Responses: 6.0

@jtimberlake
Total Positive Responses: 18.0
Total Negative Responses: 0.0
Total Neutral Responses: 13.0

@KimKardashian
Total Positive Responses: 20.0
Total Negative Responses: 1.0
Total Neutral Responses: 14.0

@selenagomez
Total Positive Responses: 14.0
Total Negative

In [54]:
aggTable = pd.DataFrame({'User': top_twitter_users, 'Positive': countPos, 'Negative': countNeg, 'Neutral': countNeutral})
aggTable.to_csv('aggregate_counts.csv')
aggTable

,Negative,Neutral,Positive,User
0,28,86,124,@katyperry
1,7,20,20,@justinbieber
2,3,12,16,@BarackObama
3,2,12,22,@taylorswift13
4,3,4,6,@rihanna
5,11,33,63,@TheEllenShow
6,3,6,12,@ladygaga
7,0,13,18,@jtimberlake
8,1,14,20,@KimKardashian
9,2,9,14,@selenagomez


In [55]:
allMentions = np.zeros((len(top_twitter_users), len(top_twitter_users)))
allPosMentions = np.zeros((len(top_twitter_users), len(top_twitter_users)))
allNeutralMentions = np.zeros((len(top_twitter_users), len(top_twitter_users)))
allNegMentions = np.zeros((len(top_twitter_users), len(top_twitter_users)))
for user in top_twitter_users:
    for tweet in good_tweets[user]:
        for recipient in tweet['recipient']:
            allMentions[twitter_users_idx[user]][twitter_users_idx[recipient]]+=1
            if tweet['sentiment']==1:
                allPosMentions[twitter_users_idx[user]][twitter_users_idx[recipient]]+=1
            elif tweet['sentiment']==-1:
                allNegMentions[twitter_users_idx[user]][twitter_users_idx[recipient]]+=1
            else:
                allNeutralMentions[twitter_users_idx[user]][twitter_users_idx[recipient]]+=1

In [56]:
DGAll=nx.DiGraph()
DGPos=nx.DiGraph()
DGNeg=nx.DiGraph()
DGNeutral=nx.DiGraph()
for i in range(len(top_twitter_users)):
    for j in range(len(top_twitter_users)):
        if allMentions[i][j]>1:
            DGAll.add_weighted_edges_from([(i,j,allMentions[i][j])])
        if allPosMentions[i][j]>1:
            DGPos.add_weighted_edges_from([(i,j,allPosMentions[i][j])])
        if allNeutralMentions[i][j]>1:
            DGNeutral.add_weighted_edges_from([(i,j,allNeutralMentions[i][j])])
        if allNegMentions[i][j]>1:
            DGNeg.add_weighted_edges_from([(i,j,allNegMentions[i][j])])

plt.clf()
nx.draw_circular(DGAll)
plt.savefig("pathAll.png")
plt.clf()
nx.draw_circular(DGPos)
plt.savefig("pathPos.png")
plt.clf()
nx.draw_circular(DGNeg)
plt.savefig("pathNeg.png")
plt.clf()
nx.draw_circular(DGNeutral)
plt.savefig("pathNeutral.png")
plt.clf()

### Saving Some Stuff ###

In [57]:
allMentionsTable = pd.DataFrame(allMentions)
allMentionsTable.columns = top_twitter_users
allMentionsTable.index = top_twitter_users
allMentionsTable.to_csv('all_mentions.csv')

allPosMentionsTable = pd.DataFrame(allPosMentions)
allPosMentionsTable.columns = top_twitter_users
allPosMentionsTable.index = top_twitter_users
allPosMentionsTable.to_csv('all_pos_mentions.csv')

allNeutralMentionsTable = pd.DataFrame(allNeutralMentions)
allNeutralMentionsTable.columns = top_twitter_users
allNeutralMentionsTable.index = top_twitter_users
allNeutralMentionsTable.to_csv('all_neutral_mentions.csv')

allNegMentionsTable = pd.DataFrame(allNegMentions)
allNegMentionsTable.columns = top_twitter_users
allNegMentionsTable.index = top_twitter_users
allNegMentionsTable.to_csv('all_neg_mentions.csv')